# Welome to Tech Day NLP Demonstration

In [3]:
import pandas as pd
import numpy as np
import time
import glob
import os

pd.options.display.max_columns = None

In [16]:
###
#    The first function
#
#    This function reads in the data from the csv file
#    It then processes the data into separate dataframes
#    These dataframes are contained within a dictionary
#    The function returns the dictionary of dataframes
#    for further processing
###

def read_data():
    df = pd.read_csv('./data/file.csv')
    
    # Initialize an empty dictionary to become the dictionary of dataframes
    d = {}    

    # rename the columns from crap into good stuff based on actual column names from csv file
    columns_good = list(df.iloc[(df[df[df.columns[1]] == 'Source'].index[0])])
    columns_bad = list(df.columns)
    cols_dict = dict(zip(columns_bad, columns_good))
    
    df.rename(columns=cols_dict, inplace=True)

    # Sanity check printing
    # print(len(columns_good))
    # print(len(columns_bad))
    # cols_dict
    
    # Find all unique sources dropping garbage at top of CSV file
    source_list = list(df.iloc[((df[df[df.columns[1]] == 'Source'].index[0])+1):,1].unique())
    
    # display(df.iloc[((df[df[df.columns[1]] == 'Source'].index[0])+1):,1])
    
    # Sanity check
    # print(source_list)    

    # create a list of the columsn that contain date/time information (used in the for loop below)
    dt_cols = ['Date(ET)', 'Time(ET)', 'LocalTime']
    
    # Certain Sources have different column structures. 
    # Create columnn lists for each source with the correct column headings
    youtube_cols = "No,Source,Link,Date,Time,Author,Author Profile,Category,Title,Description,Views,Comments,Likes,Dislikes,Favourites,Duration (seconds),Unique ID".split(sep=',')
    instagram_cols = "No,Source,Link,Date,Time,Author ID,Author Name,Language,Location,Contents,HashTags,Likes,Comments,Attachments,Brand Images,Object Images,Food Images,Scene Images,Selfie,Sentiment,Themes,Classifications,Entities,Unique ID".split(sep=',')
    allother_cols = "No,Source,Host,Link,Date(ET),Time(ET),LocalTime,Category,Author ID,Author Name,Author URL,Authority,Followers,Following,Age,Gender,Language,Country,Province/State,City,Location,Sentiment,Themes,Classifications,Entities,Alexa Rank,Alexa Reach,Title,Snippet,Contents,Summary,Bio,Unique ID,Post Source".split(sep=',')
    
    # Create a for loop to build a dictionary of dataframes
    for i in source_list:
        d['{0}'.format(i)] = df[df.iloc[:,1] == i]
        d['{0}'.format(i)].reset_index(inplace=True, drop=True)
        
        # Fix the datetime dtype issue.
        # This applies the to_datetime function to the three identified rows that contain date/time data
        # 
        # IMPORTANT
        # Because all sources were merged into ONE dataframe, some info in the date/time columns was NOT
        # date/time info. For this reason, we use 'errors='ignore''. This keeps the original data that was
        # NOT date/time data intact for when we split the data out into individual dataframes below
        for col in dt_cols:
            d['{0}'.format(i)].loc[:, col] = pd.to_datetime(d['{0}'.format(i)].loc[:, col], errors='ignore')
        
        temp_dict = {}
        
        if i == 'INSTAGRAM':
            d['{0}'.format(i)] = d['{0}'.format(i)].iloc[:,0:24]
            temp_dict = dict(zip(d['{0}'.format(i)].columns, instagram_cols))
            d['{0}'.format(i)].rename(columns=temp_dict, inplace=True)
        elif i == 'YOUTUBE':
            d['{0}'.format(i)] = d['{0}'.format(i)].iloc[:,0:17]
            temp_dict = dict(zip(d['{0}'.format(i)].columns, youtube_cols))
            d['{0}'.format(i)].rename(columns=temp_dict, inplace=True)
        else:
            temp_dict = dict(zip(d['{0}'.format(i)].columns, allother_cols))
            d['{0}'.format(i)].rename(columns=temp_dict, inplace=True)
        
    # Create a for loop to drop columns that are completely NaN in EACH dataframe
    # Also drop any rows that are completely NaN in EACH dataframe
    #for j in source_list:
    #    print(d[j].shape)    # Sanity Check
    #    d[j].dropna(axis=1, how='all', inplace=True)
    #    print(d[j].shape)    # Sanity Check
    #    d[j].dropna(axis=0, how='all', inplace=True)
    #    print(d[j].shape)    # Sanity Check
    #    d[j].reset_index(inplace=True, drop=True)
        
    # Return the dictionary of dataframes 
    return(d)

In [17]:
start = time.time()
df = read_data()
end = time.time()
print(end - start)

12.606229543685913


In [18]:
def read_data2():
    df = pd.read_csv('./data/file.csv')
    
    # Create a temporary list of sources starting from the first instance of sources 
    # and returning unique values in the 1st column (the column containing source names)
    # This also pulls in extra instances of 'Source' in the dataframe
    temp_sources = list(df.iloc[((df[df[df.columns[1]] == 'Source'].index[0])+1):,1].unique())
    
    # Create source list and use a for loop to fill the source list
    # using the temporary source list but excluding 'Source'
    source_list = []
    for i in temp_sources:
        if i != 'Source':
            source_list.append(i)
    
    # Sanity Check
    # print(source_list)
    
    # Determine the index values in the data frame where 'Source' 
    # is found in column 1. This is used later to slice the dataframe into
    # sub dataframes
    source_breaks = list(df[df[df.columns[1]] == 'Source'].index)
    
    # sanity check
    # print(source_breaks)
    
    # print(len(source_list), len(source_breaks))
    
    # create some empty holder lists and dictionaries; 'd' is going to be a dictionary of dataframes
    # The other temp lists and dictionary are used in creating these sub dataframes and associated column names
    d = {}
    bad_cols = []
    good_cols = []
    temp_dict = {}
    
    # Create a for loop to iterate through source_list to construct sub dataframes
    for i, vals in enumerate(source_list):
        
        # Create a temp var 'j' for indexing/slicing the dataframe with iloc
        j = i+1
        
        # Create an if/else to test if the value of 'j' exceeds the length of the source_list
        # If it does not, create some sub dataframes using the value of 'j' as the 'stop' point 
        # for iloc slice.
        #
        # If it does, use the length of the main dataframe as the 'stop' point for the iloc slice
        if j < len(source_list):
            d['{0}'.format(vals)] = df.iloc[source_breaks[i]+1: source_breaks[j]]
            
            # Rename the columns by creating a dictionary from the bad column names and
            # the actual column names found at a specific location in the main dataframe
            bad_cols = list(d['{0}'.format(vals)].columns)
            good_cols = list(df.iloc[source_breaks[i]])
            temp_dict = dict(zip(bad_cols, good_cols))
            
            d['{0}'.format(vals)].rename(columns=temp_dict, inplace=True)
            
        else:
            d['{0}'.format(vals)] = df.iloc[source_breaks[i]+1:]
            
            # Rename the columns by creating a dictionary from the bad column names and
            # the actual column names found at a specific location in the main dataframe
            bad_cols = list(d['{0}'.format(vals)].columns)
            good_cols = list(df.iloc[source_breaks[i]])
            temp_dict = dict(zip(bad_cols, good_cols))
            d['{0}'.format(vals)].rename(columns=temp_dict, inplace=True)
    
    for key in d.keys():
        if key == 'INSTAGRAM' or key == 'YOUTUBE':
            d[key][['Date', 'Time']] = d[key][['Date', 'Time']].apply(pd.to_datetime, errors='ignore')
        else:
            d[key][['Date(ET)', 'Time(ET)', 'LocalTime']] = d[key][['Date(ET)', 'Time(ET)', 'LocalTime']].apply(pd.to_datetime, errors='ignore')
    
    
    return(d)

In [19]:
start = time.time()
df_crazy = read_data2()
end = time.time()
print(end - start)

18.510170221328735


In [20]:
print(df_crazy.keys())

dict_keys(['NEWS', 'FORUMS', 'TUMBLR', 'TWITTER', 'INSTAGRAM'])
